<!-- Projeto-->
# <font color='blue'>Projeto</font>
## <font color='blue'>Armazenamento e Gestão de Dados com Data Lake e Data Lakehouse</font>
## <font color='blue'>Lab 8</font>
### <font color='blue'>RAG Pipeline Para IA Generativa no Databricks (com Acesso Pelo Google Colab)</font>

## Instalando e Carregando Pacotes

In [1]:
!pip install -q watermark

In [2]:
!pip install -q databricks-cli databricks-sql-connector

In [3]:
!pip install -q databricks-langchain langchain_milvus langchain-huggingface sentence-transformers beautifulsoup4

ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\inha_\\anaconda3\\Lib\\site-packages\\win32\\_win32sysloader.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
import os
os.environ["USER_AGENT"] = "LangChain (Google Colab)"

In [ ]:
# Imports
import bs4
import sentence_transformers
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from databricks_langchain import ChatDatabricks
from langchain_milvus import Milvus
import warnings
warnings.filterwarnings('ignore')

## Configurando Credenciais de Acesso ao Databricks

Crie o token conforme mostrado nas aulas.

In [ ]:
# Defina as credenciais do Databricks
os.environ["DATABRICKS_HOST"] = "url_Databricks"   # Substitua pela URL do seu Databricks
os.environ["DATABRICKS_TOKEN"] = "token"           # Substitua pelo token gerado

## Extraindo Dados da Web e Criando os Chunks

In [ ]:
# Cria o carregador de dados da web
dsa_loader = WebBaseLoader("https://blog.dsacademy.com.br/microsoft-fabric-transformando-dados-em-conhecimento/")

In [ ]:
# Executa o carregador e extrai os dados da web
documentos = dsa_loader.load()

In [ ]:
# Cria o separador de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1800, chunk_overlap = 200)

In [ ]:
# Aplica o separador e cria os chunks (documentos)
docs = text_splitter.split_documents(documentos)

In [ ]:
len(docs)

## Carregando o Modelo de Embeddings

Leia sobre modelos de embeddings no Capítulo 17 do curso.

https://huggingface.co/BAAI/bge-small-en-v1.5

In [ ]:
# Carrega o modelo de embeddings
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-small-en-v1.5")

## Criando e Carregando o Banco de Dados Vetorial

In [ ]:
# Cria o banco de dados vetorial
dsa_vector_db = Milvus.from_documents(documents = docs,
                                      embedding = embeddings,
                                      collection_name = 'dsa_collection',
                                      index_params = {"index_type": "FLAT"},
                                      connection_args = {"uri": "./milvus_dsa.db"})

In [ ]:
# Cria o retriever para recuperar os dados do Vector DB
retriever = dsa_vector_db.as_retriever()

In [ ]:
retriever

In [ ]:
# Testando o Retriever

# Define uma frase
query = "O Que é o Microsoft Fabric?"

# Busca texto similar a frase dentro do banco vetorial
dsa_vector_db.similarity_search(query, k = 1)

## Definindo o Endpoint do LLM

In [ ]:
# Definimos aqui o endpoint para o LLM no Databricks
llm = ChatDatabricks(endpoint = "databricks-dbrx-instruct", max_tokens = 200)

## Definindo o Prompt Template

In [ ]:
# Cria o texto do prompt
PROMPT_TEMPLATE = """
        Humano: Você é um assistente de IA e fornece respostas a perguntas do usuário.

        Use as seguintes informações para fornecer uma resposta concisa à pergunta entre as tags <question>.
        Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.

        <context>
        {context}
        </context>

        <question>
        {question}
        </question>

        A resposta deve ser específica e usar apenas informações confiáveis.

        Assistente:"""

In [ ]:
# Cria o prompt template
prompt = PromptTemplate(template = PROMPT_TEMPLATE, input_variables = ["context", "question"])

In [ ]:
# Cria o retriever
retriever = dsa_vector_db.as_retriever()

In [ ]:
# Função para formatar os dados
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Definindo o RAG Chain Para o RAG Pipeline

In [ ]:
# RAG Chain

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

O RunnablePassthrough no LangChain é uma classe que simplesmente passa os dados de entrada para a saída sem realizar nenhuma modificação ou processamento. É basicamente um "canal direto" para os dados. Isso é o que faremos com a questão do usuário.

## Executando o Pipeline

In [ ]:
# Invoca a chain
resposta = rag_chain.invoke(query)

In [ ]:
resposta

In [ ]:
# Nova questão
question = "Como Empresas Podem Utilizar o Microsoft Fabric?"
resposta = rag_chain.invoke(question)
resposta

In [ ]:
# Nova questão
question = "Qual o Papel do Cientista de Dados no Microsoft Fabric?"
resposta = rag_chain.invoke(question)
resposta

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim